In [176]:
# Import statements
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt
import random
import datetime as dt

from sklearn.preprocessing import StandardScaler
from tslearn.clustering import TimeSeriesKMeans
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [177]:
# Import data (Weekly Patterns and Places dataset)
df = pd.read_csv('nonPolygonData.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076020 entries, 0 to 2076019
Data columns (total 56 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   Unnamed: 0                                         int64  
 1   date_range_start                                   object 
 2   date_range_end                                     object 
 3   placekey                                           object 
 4   sg_c__parent_placekey                              object 
 5   sg_c__location_name                                object 
 6   sg_c__safegraph_brand_ids                          object 
 7   sg_c__brands                                       object 
 8   sg_c__top_category                                 object 
 9   sg_c__sub_category                                 object 
 10  sg_c__naics_code                                   float64
 11  sg_c__latitude                                    

In [179]:
# Function to change the string values in sg_wp__visits_by_day column to lists
def toList(row):
    visits = row.sg_wp__visits_by_day[1:len(row.sg_wp__visits_by_day) - 1]
    list = visits.split(",")
    return list

In [180]:
df2 = df.copy()

In [181]:
# Remove data points with null values for visits by day
df2 = df2[df2.sg_wp__visits_by_day.notna()]

In [182]:
df2

,Unnamed: 0,date_range_start,date_range_end,placekey,sg_c__parent_placekey,sg_c__location_name,sg_c__safegraph_brand_ids,sg_c__brands,sg_c__top_category,sg_c__sub_category,...,sg_wp__bucketed_dwell_times,sg_wp__related_same_day_brand,sg_wp__related_same_week_brand,sg_wp__device_type,sg_wp__normalized_visits_by_state_scaling,sg_wp__normalized_visits_by_total_visits,sg_wp__normalized_visits_by_total_visitors,sg_wp__normalized_visits_by_region_naics_visits,sg_wp__normalized_visits_by_region_naics_visitors,date
0,0,2021-04-05,2021-04-12,222-222@8gc-58v-52k,NaN,Sunoco,SG_BRAND_8d819ffb9fa80295260ecf779927ef25,Sunoco,Gasoline Stations,Gasoline Stations with Convenience Stores,...,"{""<5"":38,""5-10"":247,""11-20"":62,""21-60"":34,""61-...","{""7-Eleven"":31,""Walmart"":18,""Lowe's"":7,""McDona...","{""Walmart"":46,""7-Eleven"":38,""McDonald's"":26,""C...","{""android"":114,""ios"":211}",6694.009082,4.693066e-05,0.000137,0.000847,0.001029,2021-04-12 04:00:00+00:00
1,1,2021-07-19,2021-07-26,222-222@8gc-58v-52k,NaN,Sunoco,SG_BRAND_8d819ffb9fa80295260ecf779927ef25,Sunoco,Gasoline Stations,Gasoline Stations with Convenience Stores,...,"{""<5"":25,""5-10"":261,""11-20"":43,""21-60"":33,""61-...","{""7-Eleven"":25,""Walmart"":19,""McDonald's"":9,""Sp...","{""Walmart"":53,""7-Eleven"":34,""McDonald's"":29,""S...","{""android"":144,""ios"":171}",6723.273504,4.632336e-05,0.000133,0.000863,0.001047,2021-07-26 04:00:00+00:00
2,2,2021-03-22,2021-03-29,222-222@8gc-58v-52k,NaN,Sunoco,SG_BRAND_8d819ffb9fa80295260ecf779927ef25,Sunoco,Gasoline Stations,Gasoline Stations with Convenience Stores,...,"{""<5"":30,""5-10"":239,""11-20"":39,""21-60"":21,""61-...","{""7-Eleven"":28,""Walmart"":20,""McDonald's"":6,""Ch...","{""Walmart"":45,""7-Eleven"":35,""McDonald's"":27,""C...","{""android"":133,""ios"":164}",6047.919774,4.114717e-05,0.000129,0.000830,0.001022,2021-03-29 04:00:00+00:00
3,3,2021-03-29,2021-04-05,222-222@8gc-58v-52k,NaN,Sunoco,SG_BRAND_8d819ffb9fa80295260ecf779927ef25,Sunoco,Gasoline Stations,Gasoline Stations with Convenience Stores,...,"{""<5"":49,""5-10"":225,""11-20"":53,""21-60"":40,""61-...","{""7-Eleven"":32,""Walmart"":20,""McDonald's"":6,""Ch...","{""Walmart"":49,""7-Eleven"":37,""McDonald's"":25,""C...","{""android"":121,""ios"":187}",6662.502097,4.431645e-05,0.000132,0.000848,0.001037,2021-04-05 04:00:00+00:00
4,4,2021-02-01,2021-02-08,222-222@8gc-58v-52k,NaN,Sunoco,SG_BRAND_8d819ffb9fa80295260ecf779927ef25,Sunoco,Gasoline Stations,Gasoline Stations with Convenience Stores,...,"{""<5"":24,""5-10"":147,""11-20"":33,""21-60"":24,""61-...","{""7-Eleven"":25,""Walmart"":23,""Spinx"":7,""McDonal...","{""Walmart"":52,""7-Eleven"":35,""McDonald's"":29,""S...","{""android"":107,""ios"":96}",4213.186202,3.380348e-05,0.000089,0.000836,0.001048,2021-02-08 05:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2076015,2076015,2020-06-08,2020-06-15,zzy-222@8gd-43z-zj9,NaN,Anytime Fitness,SG_BRAND_6daa255524fe5ac244c3bed9cfbde479,Anytime Fitness,Other Amusement and Recreation Industries,Fitness and Recreational Sports Centers,...,"{""<5"":0,""5-10"":1,""11-20"":4,""21-60"":20,""61-120""...","{""Walmart"":26,""Food Lion"":9,""Sonic"":7,""Wendy's...","{""Walmart"":61,""Sonic"":27,""Chick-fil-A"":27,""Spi...","{""android"":22,""ios"":11}",837.097651,6.674504e-06,0.000014,0.000225,0.000400,2020-06-15 04:00:00+00:00
2076016,2076016,2020-04-06,2020-04-13,zzy-222@8gd-43z-zj9,NaN,Anytime Fitness,SG_BRAND_6daa255524fe5ac244c3bed9cfbde479,Anytime Fitness,Other Amusement and Recreation Industries,Fitness and Recreational Sports Centers,...,"{""<5"":0,""5-10"":2,""11-20"":0,""21-60"":0,""61-120"":...",{},{},"{""android"":4,""ios"":0}",56.841601,5.525838e-07,0.000001,0.000032,0.000059,2020-04-13 04:00:00+00:00
2076017,2076017,2020-03-23,2020-03-30,zzy-222@8gd-43z-zj9,NaN,Anytime Fitness,SG_BRAND_6daa255524fe5ac244c3bed9cfbde479,Anytime Fitness,Other Amusement and Recreation Industries,Fitness and Recreational

In [183]:
# Create new column with list values from toList()
df2['visits'] = df2.apply(toList, axis = 1)

In [184]:
df2.visits

0          [39, 52, 57, 57, 54, 67, 65]
1          [53, 50, 57, 63, 59, 55, 51]
2          [38, 45, 51, 39, 56, 61, 55]
3          [41, 47, 39, 61, 61, 82, 48]
4          [24, 36, 35, 35, 45, 29, 33]
                       ...             
2076015         [7, 5, 11, 7, 10, 8, 3]
2076016           [0, 1, 1, 0, 1, 0, 0]
2076017           [3, 1, 2, 0, 2, 0, 0]
2076018           [1, 1, 2, 1, 0, 0, 0]
2076019          [5, 8, 7, 6, 12, 9, 2]
Name: visits, Length: 2072601, dtype: object

In [185]:
# Explode lists in visit column so there are rows for each day rather than rows broken down by weeks
df3 = df2.explode('visits')

In [186]:
# Subset data into just date_range_start, longitude, latitude, location names, NAICS code, and visits
df4 = df3[['date_range_start','sg_c__longitude', 'sg_c__latitude', 'sg_c__location_name','sg_c__naics_code', 'visits']]

In [187]:
# Sort values to get a better idea of what the dataset looks like
df5 = df4.sort_values(['sg_c__location_name', 'date_range_start'])
df5

,date_range_start,sg_c__longitude,sg_c__latitude,sg_c__location_name,sg_c__naics_code,visits
1316814,2021-08-30,-79.917219,32.904885,101 Mobility,624120.0,0
1316814,2021-08-30,-79.917219,32.904885,101 Mobility,624120.0,0
1316814,2021-08-30,-79.917219,32.904885,101 Mobility,624120.0,0
1316814,2021-08-30,-79.917219,32.904885,101 Mobility,624120.0,1
1316814,2021-08-30,-79.917219,32.904885,101 Mobility,624120.0,1
...,...,...,...,...,...,...
2068561,2022-01-17,NaN,NaN,NaN,NaN,2
2068561,2022-01-17,NaN,NaN,NaN,NaN,1
2068561,2022-01-17,NaN,NaN,NaN,NaN,4
2068561,2022-01-17,NaN,NaN,NaN,NaN,3


In [188]:
# Remove data points with any null values
df6 = df5.dropna()

In [189]:
# Convert date_range_start from string to datetime
df6['date_range_start'] = pd.to_datetime(df6['date_range_start'])

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [190]:
# Add column with month to subset data into seasons
df6['month'] = df6.date_range_start.dt.month

In [191]:
# Function to subset data into seasons
def season(row):
    if row.month < 3 or row.month == 12:
        return 'winter'
    elif row.month < 6:
        return'spring'
    elif row.month < 9:
        return'summer'
    else:
        return 'fall'

In [192]:
# Apply the season function to the entire dataframe
df6['season'] = df6.apply(season, axis=1)

In [193]:
# Sort values by date_range_start and location_name to correct the dates
df7 = df6.sort_values(['date_range_start', 'sg_c__location_name']).reset_index()

In [194]:
df8 = df7.copy()

In [195]:
# Figure out the actual date for each day rather than just date_range_start/date_range_end
# and update the date column in dataframe with correct values.
for i in range(len(df8)):
    df8.at[i, 'weekday'] = i % 7
    timeToAdd = df8.at[i, 'weekday']
    df8.at[i, 'date'] = df8.at[i, 'date_range_start'] + pd.Timedelta(days=timeToAdd)

In [196]:
# Subset out the index column and location name
df9 = df8[['date','date_range_start','sg_c__longitude', 'sg_c__latitude', 'sg_c__naics_code', 'season', 'visits']]

In [197]:
# Want to compare 2019 to 2020 to 2021
df10 = df9[df9.date_range_start > pd.to_datetime('2018-12-31')]
df10['visits'] = df10['visits'].astype(int)

In [199]:
df10.to_csv('processed_data.csv', index=False)

In [117]:
# Subset data into seasons
winterS = df10[df10.season == 'winter']
springS = df10[df10.season == 'spring']
summerS = df10[df10.season == 'summer']
fallS = df10[df10.season == 'fall']

### Now for each period, I wanted to check if there was an equal number of weeks for each year, so I counted the number of unique weeks using date. Then, I looked at the number of unique weeks for each year and period, and if they differed I removed the last week because it started on the last day of the month.

#### Fall Dataset

In [ ]:
len(fallS.date.unique()) / 7

In [ ]:
# Number of fall weeks in 2019
len(fallS[fallS.date.dt.year == 2019].date.unique()) / 7

In [ ]:
# Number of fall weeks in 2020
len(fallS[fallS.date.dt.year == 2020].date.unique()) / 7

In [ ]:
# Number of fall weeks in 2021 
len(fallS[fallS.date.dt.year == 2021].date.unique()) / 7

In [ ]:
# These numbers are equal, so no weeks need to be removed from the dataset
fallSClean = fallS

#### Spring Dataset

In [ ]:
len(springS.date.unique()) / 7

In [ ]:
len(springS[springS.date.dt.year == 2019].date.unique()) / 7

In [ ]:
len(springS[springS.date.dt.year == 2020].date.unique()) / 7

In [ ]:
len(springS[springS.date.dt.year == 2021].date.unique()) / 7

In [ ]:
springS[springS.date.dt.year == 2021].date.unique()
# Need to remove 5-31 week because it adds an extra week to 2021 Spring and brings June dates into Spring

In [ ]:
# Remove the week of May 31, 2021
springSClean = springS[springS['date'] < pd.to_datetime('2021-05-31T00:00:00.000000000')]

#### Summer Dataset

In [41]:
len(summerS.date.unique()) / 7

40.0

In [42]:
len(summerS[summerS.date.dt.year == 2019].date.unique()) / 7

13.0

In [43]:
len(summerS[summerS.date.dt.year == 2020].date.unique()) / 7

14.0

In [44]:
len(summerS[summerS.date.dt.year == 2021].date.unique()) / 7

13.0

In [45]:
summerS[summerS.date.dt.year == 2020].date.unique()
# Remove August 31 from the dataset because it adds an extra week to Summer 2021

array(['2020-06-01T00:00:00.000000000', '2020-06-02T00:00:00.000000000',
       '2020-06-03T00:00:00.000000000', '2020-06-04T00:00:00.000000000',
       '2020-06-05T00:00:00.000000000', '2020-06-06T00:00:00.000000000',
       '2020-06-07T00:00:00.000000000', '2020-06-08T00:00:00.000000000',
       '2020-06-09T00:00:00.000000000', '2020-06-10T00:00:00.000000000',
       '2020-06-11T00:00:00.000000000', '2020-06-12T00:00:00.000000000',
       '2020-06-13T00:00:00.000000000', '2020-06-14T00:00:00.000000000',
       '2020-06-15T00:00:00.000000000', '2020-06-16T00:00:00.000000000',
       '2020-06-17T00:00:00.000000000', '2020-06-18T00:00:00.000000000',
       '2020-06-19T00:00:00.000000000', '2020-06-20T00:00:00.000000000',
       '2020-06-21T00:00:00.000000000', '2020-06-22T00:00:00.000000000',
       '2020-06-23T00:00:00.000000000', '2020-06-24T00:00:00.000000000',
       '2020-06-25T00:00:00.000000000', '2020-06-26T00:00:00.000000000',
       '2020-06-27T00:00:00.000000000', '2020-06-28

In [46]:
# Remove the week of August 31, 2020
summerCond1 = summerS['date'] < pd.to_datetime('2020-08-31T00:00:00.000000000')
summerCond2 = summerS['date'] > pd.to_datetime('2021-01-01T00:00:00.000000000')
summerSClean = summerS[summerCond1 | summerCond2]

### The winter dataset is different because it includes two different years (December - February)

- winter2019 = December 2019 to February 2020
- winter2020 = December 2020 to February 2021
- winter2021 = December 2021 to February 2022 - We don't have full data on this portion

In [ ]:
# To get the random weeks to train the k-Means clustering algorithm, 
# I randomly chose any week but the last week in each period (so
# the following week could be used for testing).
fallWeek = random.choices(range(12))[0]
fallWeekTest = fallWeek + 1
springWeek = random.choices(range((12)))[0]
springWeekTest = springWeek + 1
summerWeek = random.choices(range((12)))[0]
summerWeekTest = summerWeek + 1
winterWeek = random.choices(range(12))[0]
winterWeekTest = winterWeek + 1

In [209]:
# Subsetting out the fall training and testing data
fallWeekDate = fallSClean.date_range_start.unique()[fallWeek]
fallWeekData = fallSClean[fallSClean.date_range_start == fallWeekDate].reset_index(drop = True)
fallWeekTestDate = fallSClean.date_range_start.unique()[fallWeekTest]
fallWeekTestData = fallSClean[fallSClean.date_range_start == fallWeekTestDate].reset_index(drop = True)

# Subsetting out the spring training and testing data
springWeekDate = springSClean.date_range_start.unique()[springWeek]
springWeekData = springSClean[springSClean.date_range_start == springWeekDate].reset_index(drop = True)
springWeekTestDate = springSClean.date_range_start.unique()[springWeekTest]
springWeekTestData = springSClean[springSClean.date_range_start == springWeekTestDate].reset_index(drop = True)

# Subsetting out the summer training and testing data
summerWeekDate = summerSClean.date_range_start.unique()[summerWeek]
summerWeekData = summerSClean[summerSClean.date_range_start == summerWeekDate].reset_index(drop = True)
summerWeekTestDate = summerSClean.date_range_start.unique()[summerWeekTest]
summerWeekTestData = summerSClean[summerSClean.date_range_start == summerWeekTestDate].reset_index(drop = True)

winterWeekDate = winterSClean.date_range_start.unique()[fallWeek]
winterWeekData = winterSClean[winterSClean.date_range_start == winterWeekDate].reset_index(drop = True)
winterWeekTestDate = winterSClean.date_range_start.unique()[winterWeekTest]
winterWeekTestData = winterSClean[winterSClean.date_range_start == winterWeekTestDate].reset_index(drop = True)

NameError: name 'winterSClean' is not defined

In [49]:
summerWeekData

,date,date_range_start,sg_c__longitude,sg_c__latitude,sg_c__naics_code,season,visits
0,2019-08-05,2019-08-05,-79.948399,32.728500,531120.0,summer,<bound method Rolling.median of Rolling [windo...
1,2019-08-06,2019-08-05,-79.948399,32.728500,531120.0,summer,<bound method Rolling.median of Rolling [windo...
2,2019-08-07,2019-08-05,-79.948399,32.728500,531120.0,summer,<bound method Rolling.median of Rolling [windo...
3,2019-08-08,2019-08-05,-79.948399,32.728500,531120.0,summer,<bound method Rolling.median of Rolling [windo...
4,2019-08-09,2019-08-05,-79.948399,32.728500,531120.0,summer,<bound method Rolling.median of Rolling [windo...
...,...,...,...,...,...,...,...
66194,2019-08-07,2019-08-05,-80.147438,32.607367,448140.0,summer,<bound method Rolling.median of Rolling [windo...
66195,2019-08-08,2019-08-05,-80.147438,32.607367,448140.0,summer,<bound method Rolling.median of Rolling [windo...
66196,2019-08-09,2019-08-05,-80.147438,32.607367,448140.0,summer,<bound method Rolling.median of Rolling [windo...
66197,2019-08-10,2019-08-05,-80.147438,32.607367,448140.0,summer,<bound method Rolling.median of Rolling [windo...


In [50]:
# Removing irrelevant columns such as date_range_start, sg_c__naics_code, and season to
# reduce dimension of data for k-means algorithm
springWeekData2 = springWeekData[['date', 'sg_c__longitude', 'sg_c__latitude', 'visits']]
springTestData2 = springWeekTestData[['date', 'sg_c__longitude', 'sg_c__latitude', 'visits']]

In [51]:
# Convert date to ordinal rather than datetime so k-means works
springWeekData2.date = springWeekData2.date.map(dt.datetime.toordinal)
springTestData2.date = springTestData2.date.map(dt.datetime.toordinal)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


## K-Means Clustering Algorithm

In [52]:
# To use the TSLearn k-Means clustering algorithm, first you have 
# to scale the data.
scaler = StandardScaler()


In [151]:
def k_meansSeasonAlgorithm(seasonData, seasonTestData):
    # Fit the scaler to the x_train data
    scaler.fit(seasonData.drop('visits', axis = 1))
    train_scaled_features = pd.DataFrame(scaler.transform(seasonData.drop('visits', axis = 1)))
    x_train = train_scaled_features
    y_train = seasonData.visits
    
    # Fit the scaler to the x_test data
    scaler.fit(seasonTestData.drop('visits', axis = 1))
    # Scale the x_test data
    test_scaled_features = pd.DataFrame(scaler.transform(seasonTestData.drop('visits', axis = 1)))
    
    x_test = test_scaled_features
    y_test = seasonTestData.visits
    
    # Build the model.
    # DTW is a common metric for time series k-means.
    model = TimeSeriesKMeans(n_clusters=6, metric="dtw")
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    print(confusion_matrix(y_test.astype(str), predictions.astype(str)))
    print(classification_report(y_test.astype(str), predictions.astype(str), zero_division = 0))
        

In [152]:
# This algorithm is not actually useful for this type of data because it wants one dimensional data, assuming that it is a time series
k_meansSeasonAlgorithm(springWeekData2, springTestData2)

/opt/anaconda3/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 67326 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/opt/anaconda3/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 67326 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/opt/anaconda3/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 67326 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/opt/anaconda3/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 67326 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '
/opt/anaconda3/lib/python3.8/site-packages/tslearn/utils/utils.py:88: UserWarning: 2-Dimensional dat